In [ ]:
# Selenium库巨潮资讯网评估说明批量下载
## 巨潮资讯网是中国证监会指定的上市公司信息披露网站,平台提供上市公司公告、公司资讯、公司互动、股东大会网络投票等内容功能。也包括披露的评估报告、说明等

In [ ]:
# 导入库
## from-import从selenium自动检测库中导入其模拟浏览器控制功能
from selenium import webdriver
from selenium.webdriver.common.by import By
## re正则表达式库用于后面数据清洗
import re
## time库用于设置等待时间
import time
## os库用于系统文件操作方便存储文件
import os

In [ ]:
## 使用Selenium库提取数据，还需要安装一下模拟浏览器。
## 本次使用Google浏览器Chrome,其模拟浏览器为ChromeDriver。
## 模拟浏览器需要与浏览器版本一致，在Google浏览器菜单栏选择帮助->关于Google Chrome可查看浏览器版本号
## 对应的模拟浏览器为ChromeDriver
## 在https://sites.google.com/a/chromium.org/chromedriver/downloads或镜像网站http://npm.taobao.org/mirrors/chromedriver/下载对应版本安装包
## 解压后将其exe文件放在Python\scripts文件夹中，在命令提示符中输入where python可查到python文件夹位置

In [ ]:
# 获取下载页面信息
## Selenium库链接并模拟访问
### 设置模拟浏览器webdriver.Chrome()和url地址
browser = webdriver.Chrome()
url = 'http://www.cninfo.com.cn/new/fulltextSearch?notautosubmit=&keyWord=评估说明'
### .get(url)启动模拟浏览器并进入url地址
browser.get(url)
### time库延时等待页面读取
time.sleep(5)
### .page_source提取页面数据
data = browser.page_source
## 点击翻页并存储数据
### 建立一个列表存储每页信息
datas = []
datas.append(data) 
## 设置翻页次数并执行翻页
pages=2
for i in range(pages):  
## 在页面f12后用箭头查看按钮代码，右键copy->copy Xpath查询xpath地址
## browser.find_element_by_xpath查找xpath地址，.click()模拟鼠标点击
    browser.find_element(By.XPATH,'//*[@id="fulltext-search"]/div[2]/div/div/div[2]/div[4]/div[2]/div/button[2]/i').click()
## 等待加载
    time.sleep(3)
## 获取下页信息
    data = browser.page_source
    datas.append(data)
    time.sleep(1)
## 使用.join命令将list中信息链接起来，前面""中为连接符设置，这里不需要故设置为空
alldata = "".join(datas)
### 推出模拟浏览器
browser.quit()

In [ ]:
#alldata
#<span title="" class="r-title">招商局蛇口工业区控股股份有限公司拟发行股份购买深圳市投资控股有限公司持有的深圳市南油（集团）有限公司24%股权涉及的深圳市南油（集团）有限公司股东全部权益价值资产<em>评估</em><em>说明</em></span> 

In [ ]:
# re库数据清洗
## 查找标题，网址，日期信息
####这里我们只需要记住(.*?)为取()中的任意内容
p_title = '<span title="" class="r-title">(.*?)</span>'
p_href = '<a target="_blank" href="(.*?)" data-id='
title = re.findall(p_title, alldata)
print(title)
href = re.findall(p_href, alldata)
print(href)


In [ ]:
## 对上面查到信息进行清洗
for i in range(len(title)):
## re.sub,re库替换命令，将<>部分去掉
    title[i] = re.sub('<.*?>', '', title[i])
## 补上主域名去掉amp;
    href[i] = 'http://www.cninfo.com.cn' + href[i]
    href[i] = re.sub('amp;', '', href[i])
    print(str(i + 1) + '.' + title[i])
    print(href[i])

In [ ]:
# 进一步筛选评估说明字样是否在其中
for i in range(len(title)):
    if '评估说明' in title[i]:  
        title[i] = title[i]
        href[i] = href[i]
    else:
        title[i] = ''
        href[i] = ''
while '' in title:
    title.remove('')
while '' in href:
    href.remove('')

In [ ]:
# 逐页下载
# for i in range(len(href)):
for i in range(5):   ##限定下载数量
## 设置下载路径(目录需要提前建立好)
    chrome_options = webdriver.ChromeOptions()
    prefs = {'profile.default_content_settings.popups': 0,
             'download.default_directory': 'c:\\pinggushuoming'} 
    chrome_options.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome(chrome_options=chrome_options)
## 进入之前取得的下载链接
    browser.get(href[i])
## 寻找下载按键并点击
    browser.find_element(By.XPATH,
        '//*[@id="noticeDetail"]/div/div[1]/div[3]/div[1]/button/span').click() 
    time.sleep(6) 
## 判断是否下载完成，设置一个download_status下载状态标志进行检测
    download_status = False
    while not download_status:  
        download_status = True
## 报告文件没下载完时后缀名为.crdownload
## 用os库的.listdir命令检索文件夹中所有文件
        for j in os.listdir('c:\\pinggushuoming'):
## 用.endswith命令查询文件中是否有没下载完的.crdownload文件名，如果没有就把下载状态标志改为True结束循环判断
            if j.endswith(".crdownload"):
                download_status = False
                time.sleep(5)
## 文字工作提示
    print(str(i+1) + '.' + title[i] + '下载完毕')
    browser.quit()
